In [270]:
import numpy as np
import itertools as itr

import os as os
import sys as sys 
import pandas as pd
import warnings
import time as time
import random

sys.path.append(os.path.join("..","Libraries","QML_lib"))
import Evo as evo
import DataBase 
import QMD
import QML
import ModelGeneration
import BayesF
import matplotlib.pyplot as plt

global paulis_list
paulis_list = {'i' : np.eye(2), 'x' : evo.sigmax(), 'y' : evo.sigmay(), 'z' : evo.sigmaz()}

warnings.filterwarnings("ignore", message='Negative weights occured', category=RuntimeWarning)

from __future__ import print_function # so print doesn't show brackets


class NewQMDClass():
    #TODO: rename as ModelsDevelopmentClass when finished
    def __init__(self, initial_op_list, true_operator='x', maxmodnum=3, gaussian=True):
#    def __init__(self, initial_op_list, true_op_list, true_param_list):

        trueOp = DataBase.operator(true_operator)
        self.InitialOpList = initial_op_list
        self.TrueOpList = trueOp.constituents_operators
        self.TrueParamsList = [random.random() for i in self.TrueOpList]
        #self.TrueHam = evo.getH(self.TrueParamsList, self.TrueOpList)
        self.TrueHam = np.tensordot(self.TrueParamsList, self.TrueOpList, axes=1)
        self.initiateDB()
        self.MaxModNum = maxmodnum #TODO: necessary?
        self.gaussian = gaussian
        
        print("QMD has true params : ")
        print(self.TrueParamsList)
        print("And true operators: ")
        print(self.TrueOpList)
        
    def initiateDB(self):
        self.db, self.legacy_db, self.model_lists = \
            DataBase.launch_db(
                gen_list = self.InitialOpList,
                true_ops = self.TrueOpList,
                true_params = self.TrueParamsList
            )
    def addModel(self, model, treeID=0):
        DataBase.add_model(
            model_name = model,
            running_database = self.db,
            model_lists = self.model_lists,
            true_ops = self.TrueOpList,
            true_params = self.TrueParamsList,
            treeID = treeID
        )
            
            
    def getModelInstance(self, name):
        try: 
            instance = DataBase.get_qml_instance(self.db, name)
            return instance
        except: 
            if name in list(self.legacy_db['<Name>']):
                print("Operator in legacy databse - retired. ")
            else: 
                print("Model not found.")
    def getOperatorInstance(self, name):
        try: 
            return DataBase.get_operator_instance(self.db, name)
        except:
            if name in list(self.legacy_db['<Name>']):
                print("Operator in legacy databse - retired. ")
            else: 
                print("Operator not found.")

    def getModelDBIndex(self, name):
        return DataBase.get_location(self.db, name)
                
    def killModel(self, name):
        if name not in list(self.db['<Name>']):
            print("Cannot remove ", name, "; not in ", list(self.db["<Name>"]))
        else:
            print("Killing model", name)
            # Add to legacy_db
            DataBase.move_to_legacy(self.db, self.legacy_db, name)
            model_instance = self.getModelInstance(name)
            operator_instance = self.getOperatorInstance(name)
            # Remove from self.db
            self.db = DataBase.remove_model(self.db, name)
            del model_instance
            del operator_instance
    
            #TODO: plot?

    def run_iqle(self, model, n_exp=50):
        model_exists=False
        if model in list(self.db['<Name>']):
            model_exists = True
        elif model in list(self.legacy_db['<Name>']):
            print("Model ", model, " previously considered and retired.")
            
        if model_exists: 
            model_instance = self.getModelInstance(model)
            model_instance.UpdateModel(n_exp)
            #model_instance.BayesOnModelsWithinTrees
            kill_this_model = self.killModelTest(model) # return whether to kill this model or make it remain active but saturated, etc.
            if(kill_this_model):
                self.killModel(model)
        else: 
            print("Model ", model ,"does not exist")
        
    def killModelTest(self, name):
        # should compute Bayes w/ models in same tree
        # return True to kill, else False 
        if random.choice([0,1]) > 0.5:
            return False 
        else:
            return False
        
    def updateModelRecord(self, name, field, new_value=None, increment=None):
        DataBase.update_field(
            self.db, 
            name,
            field,
            new_value=new_value,
            increment=increment
        )
    def pullField(self, name, field):
        return DataBase.pull_field(self.db, name, field)
    
    
    def BayesFactorsTreeLevel(self, treeID):
        # Add 1 to BayesSum in qmd.db when model "wins" a Bayes factor 
        tree_neighbours_names = list(qmd.db[qmd.db['TreeID']==treeID]['<Name>'])
        tree_neighbours_models = list(qmd.db[qmd.db['TreeID']==treeID]['Model_Class_Instance'])

        bayes_high = 10
        bayes_low = 0.1
        for i in range(len(tree_neighbours_names)):
            model_a = tree_neighbours_models[i]
            for j in range(i+1, len(tree_neighbours_names)):
                    model_b = tree_neighbours_models[j]
                    bayes_factor = BayesF.BayesFactorCalc(model_a, model_b)
                    if bayes_factor > bayes_high: 
                        winner = tree_neighbours_names[i]
                        self.updateModelRecord(winner, field='BayesSum', increment=1)
                    elif bayes_factor < bayes_low : 
                        winner = tree_neighbours_names[j]
                        self.updateModelRecord(winner, field='BayesSum', increment=1)

    def championByTreeName(self, treeID):
    
        champions = list(self.db[self.db['TreeID']==treeID].sort_values('BayesSum', ascending=True).max(axis=1))
        if len(champions) != 1:
            print("No distinct champion. List returned.")
        else: 
            print("Champion of tree", treeID, " is ", champion)
        return champions
    
    def championByTreeModel(self, treeID):
        return self.db[self.db['TreeID']==treeID].sort_values('BayesSum', ascending=True).max()['Model_Class_Instance']

                    
    def floorRule(self):
        # Kill all models which have no Bayes points
        pointlessBayes = list(self.db[self.db['BayesSum']==0]['<Name>'])
        for name in pointlessBayes: 
            self.killModel(name)
                    
    def championOfChampions(self):
        activeTrees = list(self.db['TreeID'].unique()) # unique tree id list
        championsModelList = []
        championsNameList = []
        for tree in activeTrees: 
            championsModelList.append(self.championByTreeModel(treeID=tree))
            championsNameList.append(self.championByTreeName(treeID=tree))

        bayes_low = 0.1 # threshold 
        bayes_high = 10
        winning_model = championsModelList[0]
        winning_model_name = championsNameList[0]
        winner_changed = False
        for i in range(1, len(championsNameList)):
            model_i = championsModelList[i]
            bayes_factor = BayesF.BayesFactorCalc(winning_model, model_i)
            print("Bayes factor between", winning_model_name, " and ", championsNameList[i], " = ", bayes_factor)
            if bayes_factor < bayes_low : #i.e. model_i is better
                winning_model = model_i
                winning_model_name = championsNameList[i]
                winner_changed = True
            elif bayes_factor > bayes_high: 
                winner_changed = True
        if winner_changed: 
            print("Champion of champions is model ", winning_model_name)
            return winning_model
        else:
            print("No outright champion. List of tree-level champions returned instead.")
            return championsNameList
   
    def championRule(self):
        activeTrees = list(self.db['TreeID'].unique())
        for tree in activeTrees: 
            self.BayesFactorsTreeLevel(treeID=tree)
        
        if len(activeTrees) == 1:
            tree = activeTrees[0]
            print("Only one tree. Best model is ", self.championByTreeName(treeID=tree))
            return self.championByTreeName(treeID=tree)

        else: 
            return self.championOfChampions()

In [272]:
# generators = ['x', 'xTx', 'xTxTTx'] # Ising up to 3 spins
generators = ['x', 'y', 'z']
true_op = random.choice(generators)

generators = ['x', 'y']
true_op = 'z'

print("True op = ", true_op)
qmd= NewQMDClass(initial_op_list=generators, true_operator=true_op)

new_tree_models = ['xPy', 'yPz']
for model in new_tree_models:
    qmd.addModel(model, treeID=1)

new_tree_models = ['zPx', 'yPzPx', 'zPy']
for model in new_tree_models:
    qmd.addModel(model, treeID=2)

    
new_tree_models = ['xPi', 'zPi', 'iMz']
for model in new_tree_models:
    qmd.addModel(model, treeID=3)    
    
for model_name in qmd.db['<Name>']:
    print("Running IQLE for model ", model_name)
    qmd.run_iqle(model_name, n_exp = 40) 
    
qmd.championRule()  
    

True op =  z
Initialization Ready
Initialization Ready
('Model ', 'x', ' not previously considered -- adding.')
Initialization Ready
('Model ', 'y', ' not previously considered -- adding.')
Initialization Ready
QMD has true params : 
[0.0738367536132305]
And true operators: 
[array([[ 1.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j]])]
('Model ', 'xPy', ' not previously considered -- adding.')
Initialization Ready
('Model ', 'yPz', ' not previously considered -- adding.')
Initialization Ready
('Model ', 'zPx', ' not previously considered -- adding.')
Initialization Ready
('Model ', 'yPzPx', ' not previously considered -- adding.')
Initialization Ready
('Model', 'yPz', ' previously considered at location', 3)
('Model ', 'xPi', ' not previously considered -- adding.')
Initialization Ready
('Model ', 'zPi', ' not previously considered -- adding.')
Initialization Ready
('Model ', 'iMz', ' not previously considered -- adding.')
Initialization Ready
Running IQLE for model  x
Initial time selected

/usr/local/lib/python2.7/dist-packages/qinfer/smc.py:285: ApproximationWarning: Extremely small n_ess encountered (1.00008929481). Resampling is likely to fail. Consider adding particles, or resampling more often.
  ApproximationWarning


Final time selected > 977.032733609
Final Parameters mean and stdev:[  5.12083361e-01   3.41676107e-04]
Final Parameters mean and stdev:[  2.14903231e-01   8.13683195e-05]
Running IQLE for model  zPi
Initial time selected > 1.09142340663
Final time selected > 1.22644716054
Final Parameters mean and stdev:[ 0.17301198  0.13654439]
Final Parameters mean and stdev:[ 0.53035085  0.57399338]
Running IQLE for model  iMz
Initial time selected > 0.728729523698
Final time selected > 20.4421391023
Final Parameters mean and stdev:[ 0.10078561  0.0492905 ]
No distinct champion. List returned.
No distinct champion. List returned.
No distinct champion. List returned.
No distinct champion. List returned.
Bayes factor between ['x', 'y']  and  ['yPz', 'xPy']  =  0.00858463241436
Bayes factor between ['yPz', 'xPy']  and  ['yPzPx', 'zPx']  =  4048.63779301
Bayes factor between ['yPz', 'xPy']  and  ['iMz', 'zPi', 'xPi']  =  8.42198881745e-08
Champion of champions is model  ['iMz', 'zPi', 'xPi']


<QML.ModelLearningClass instance at 0x7f19d006a680>

In [273]:
qmd.db

,<Name>,Alph_Name,BayesSum,Epoch_Start,Model_Class_Instance,Operator_Instance,Selected,Status,TreeID
0,x,x,0.0,0.0,<QML.ModelLearningClass instance at 0x7f199621...,<DataBase.operator instance at 0x7f1996215710>,0.0,Ready,0.0
1,y,y,1.0,0.0,<QML.ModelLearningClass instance at 0x7f19961e...,<DataBase.operator instance at 0x7f19d00e89e0>,0.0,Ready,0.0
2,xPy,xPy,1.0,0.0,<QML.ModelLearningClass instance at 0x7f19d005...,<DataBase.operator instance at 0x7f19d0f85440>,0.0,Ready,1.0
3,yPz,yPz,0.0,0.0,<QML.ModelLearningClass instance at 0x7f199612...,<DataBase.operator instance at 0x7f19d0092b90>,0.0,Ready,1.0
4,zPx,xPz,1.0,0.0,<QML.ModelLearningClass instance at 0x7f19933d...,<DataBase.operator instance at 0x7f19933d27e8>,0.0,Ready,2.0
5,yPzPx,xPyPz,0.0,0.0,<QML.ModelLearningClass instance at 0x7f199927...,<DataBase.operator instance at 0x7f19d07bf638>,0.0,Ready,2.0
6,xPi,iPx,2.0,0.0,<QML.ModelLearningClass instance at 0x7f19d006...,<DataBase.operator instance at 0x7f19992eb128>,0.0,Ready,3.0
7,zPi,iPz,1.0,0.0,<QML.ModelLearningClass instance at 0x7f19d006...,<DataBase.operator instance at 0x7f19d006a710>,0.0,Ready,3.0
8,iMz,iMz,0.0,0.0,<QML.ModelLearningClass instance at 0x7f199442...,<DataBase.operator instance at 0x7f19d0802950>,0.0,Ready,3.0


In [271]:
# generators = ['x', 'xTx', 'xTxTTx'] # Ising up to 3 spins
generators = ['iMz', 'z']
true_op = 'z'
qmd= NewQMDClass(initial_op_list=generators, true_operator=true_op)

for model_name in qmd.db['<Name>']:
    print("Running IQLE for model ", model_name)
    qmd.run_iqle(model_name, n_exp = 10) 
qmd.championRule()  

    

Initialization Ready
Initialization Ready
('Model ', 'iMz', ' not previously considered -- adding.')
Initialization Ready
('Model ', 'z', ' not previously considered -- adding.')
Initialization Ready
QMD has true params : 
[0.8082886280985719]
And true operators: 
[array([[ 1.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j]])]
Running IQLE for model  iMz
Initial time selected > 2.24635745322
Final time selected > 107.277567574
Final Parameters mean and stdev:[ 0.71848541  0.23863172]
Running IQLE for model  z
Initial time selected > 0.862832080195
Final time selected > 6.35283207937
Final Parameters mean and stdev:[ 0.63259615  0.3577834 ]
No distinct champion. List returned.
Only one tree. Best model is  ['iMz', 'z']
No distinct champion. List returned.


['iMz', 'z']

In [274]:
qmd.db

,<Name>,Alph_Name,BayesSum,Epoch_Start,Model_Class_Instance,Operator_Instance,Selected,Status,TreeID
0,x,x,0.0,0.0,<QML.ModelLearningClass instance at 0x7f199621...,<DataBase.operator instance at 0x7f1996215710>,0.0,Ready,0.0
1,y,y,1.0,0.0,<QML.ModelLearningClass instance at 0x7f19961e...,<DataBase.operator instance at 0x7f19d00e89e0>,0.0,Ready,0.0
2,xPy,xPy,1.0,0.0,<QML.ModelLearningClass instance at 0x7f19d005...,<DataBase.operator instance at 0x7f19d0f85440>,0.0,Ready,1.0
3,yPz,yPz,0.0,0.0,<QML.ModelLearningClass instance at 0x7f199612...,<DataBase.operator instance at 0x7f19d0092b90>,0.0,Ready,1.0
4,zPx,xPz,1.0,0.0,<QML.ModelLearningClass instance at 0x7f19933d...,<DataBase.operator instance at 0x7f19933d27e8>,0.0,Ready,2.0
5,yPzPx,xPyPz,0.0,0.0,<QML.ModelLearningClass instance at 0x7f199927...,<DataBase.operator instance at 0x7f19d07bf638>,0.0,Ready,2.0
6,xPi,iPx,2.0,0.0,<QML.ModelLearningClass instance at 0x7f19d006...,<DataBase.operator instance at 0x7f19992eb128>,0.0,Ready,3.0
7,zPi,iPz,1.0,0.0,<QML.ModelLearningClass instance at 0x7f19d006...,<DataBase.operator instance at 0x7f19d006a710>,0.0,Ready,3.0
8,iMz,iMz,0.0,0.0,<QML.ModelLearningClass instance at 0x7f199442...,<DataBase.operator instance at 0x7f19d0802950>,0.0,Ready,3.0


In [277]:
treeID=0
list(qmd.db[qmd.db['TreeID']==treeID].sort_values('BayesSum', ascending=True).max()['<Name>']

SyntaxError: invalid syntax (<ipython-input-277-38079ee057c1>, line 2)

In [296]:
qmd.db['<Name>']=='x'

0     True
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
Name: <Name>, dtype: bool

In [287]:
qmd.db.groupby([qmd.db['TreeID']==treeID]).max()


,<Name>,Alph_Name,BayesSum,Epoch_Start,Model_Class_Instance,Operator_Instance,Selected,Status,TreeID
TreeID,,,,,,,,,
False,zPx,yPz,2.0,0.0,<QML.ModelLearningClass instance at 0x7f19d006...,<DataBase.operator instance at 0x7f19d0f85440>,0.0,Ready,3.0
True,y,y,1.0,0.0,<QML.ModelLearningClass instance at 0x7f199621...,<DataBase.operator instance at 0x7f19d00e89e0>,0.0,Ready,0.0


In [228]:
treeID=1
qmd.db[qmd.db['TreeID']==treeID].sort_values('BayesSum', ascending=True)

,<Name>,Alph_Name,BayesSum,Epoch_Start,Model_Class_Instance,Operator_Instance,Selected,Status,TreeID
3,xPy,xPy,0.0,0.0,<QML.ModelLearningClass instance at 0x7f199926...,<DataBase.operator instance at 0x7f19d0f88cf8>,0.0,Ready,1.0
4,yPz,yPz,0.0,0.0,<QML.ModelLearningClass instance at 0x7f19961f...,<DataBase.operator instance at 0x7f19961fdd88>,0.0,Ready,1.0


In [215]:
print("test")

test


In [201]:
qmd.db

,<Name>,Alph_Name,BayesSum,Epoch_Start,Model_Class_Instance,Operator_Instance,Selected,Status,TreeID
0,x,x,2.0,0.0,<QML.ModelLearningClass instance at 0x7f19a513...,<DataBase.operator instance at 0x7f19d00ccb00>,0.0,Ready,0.0
1,y,y,0.0,0.0,<QML.ModelLearningClass instance at 0x7f19a50f...,<DataBase.operator instance at 0x7f19d0f9a7a0>,0.0,Ready,0.0
2,z,z,1.0,0.0,<QML.ModelLearningClass instance at 0x7f199926...,<DataBase.operator instance at 0x7f1999264ea8>,0.0,Ready,0.0
3,xPy,xPy,0.0,0.0,<QML.ModelLearningClass instance at 0x7f19d072...,<DataBase.operator instance at 0x7f19d07871b8>,0.0,Ready,1.0
4,yPz,yPz,1.0,0.0,<QML.ModelLearningClass instance at 0x7f19a5f2...,<DataBase.operator instance at 0x7f19a498cef0>,0.0,Ready,1.0


In [200]:
qmd.championOfChampions()

('Bayes factor between', 'z', ' and ', 'yPz', ' = ', 9.0851433427648072)
('Champion of champions is model ', 'z')


<QML.ModelLearningClass instance at 0x7f19a513ebd8>

In [191]:
ypz_instance = qmd.getModelInstance('yPz')

In [192]:
ypz_instance.FinalParams

array([[ 0.78811552,  0.39000862],
       [ 0.17338596,  0.47168889]])

In [175]:
activeTrees = list(qmd.db['TreeID'].unique())
championsModelList = []
championsNameList = []
for tree in activeTrees: 
    championsModelList.append(championByTreeModel(treeID=tree))
    championsNameList.append(championByTreeName(treeID=tree))

    
winning_model = championsModelList[0]
winning_model_name = championsNameList[0]
for i in range(1, len(championsNameList)):
    model_i = championsModelList[i]
    bayes_factor = BayesF.BayesFactorCalc(winning_model, model_i)
    if bayes_factor < 1 : #i.e. model_i is better
        winning_model = model_i
        winning_model_name = championsNameList[i]

print("Champion of champions is model ", winning_model_name)


NameError: name 'championByTreeModel' is not defined

In [19]:
tree_neighbours_models

[<QML.ModelLearningClass instance at 0x7f19d0f9a200>,
 <QML.ModelLearningClass instance at 0x7f19d0f881b8>,
 <QML.ModelLearningClass instance at 0x7f19d0f885a8>,
 <QML.ModelLearningClass instance at 0x7f19d0f9a368>]

In [ ]:
qmd.TrueOpList

t_op = [evo.sigmax()]

In [ ]:
def update_field(db, name, field, new_value=None, increment=None):
    idx = DataBase.get_location(db, name)
    if new_value is not None:
        db.loc[idx, field] = new_value
    elif increment is not None:
        db.loc[idx, field] = db.loc[idx, field]+ 1
    else: 
        print("Must pass new_value or increment")

In [ ]:
db, legacy_db, model_lists = DataBase.launch_db(
        gen_list=['x', 'y'],
        true_ops = t_op,
        true_params=[0.2]
    )

In [ ]:
idx=DataBase.get_location(db, 'x')

In [ ]:
db.loc[0,'<Name>'] = 'x_new'

In [ ]:
db

In [ ]:
update_field(db, 'x', field='Epoch_Start', new_value=10)

In [ ]:
db

In [ ]:
idx=qmd.getModelDBIndex('i')

In [ ]:
qmd.db[qmd.db['TreeID']==0]

In [ ]:
to_try=true_op
#to_try='x'
qmd.addModel(to_try)
qmd.run_iqle(to_try, n_exp = 80)

In [ ]:
x_inst = qmd.getModelInstance('x')
y_inst = qmd.getModelInstance('y')

In [ ]:
import BayesF as Bayes

In [ ]:
Bayes.BayesFactorCalc(x_inst, y_inst)

In [ ]:
Bayes.BayesFactorCalc(y_inst, x_inst)

## Doesn't work when you try a 2 qubit model on a 1-qubit true model

In [ ]:
to_try='xTx'
qmd.addModel(to_try)
qmd.run_iqle(to_try, n_exp = 80)

## But it works when the correct size system is used

In [ ]:
true_op = 'xTxPPyTi'
qmd= NewQMDClass(initial_op_list=generators, true_operator=true_op)
to_try=true_op
qmd.addModel(to_try)
qmd.run_iqle(to_try, n_exp = 80)